<a href="https://colab.research.google.com/github/Mgalvaz/license_plate-recognizer/blob/main/notebooks/train_LPD_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
from tqdm.notebook import tqdm
import torch
from torch import nn
from torch.functional import F
from torch.optim import AdamW
from torch.utils.data import DataLoader
from torchvision.ops import box_iou, nms
from torchvision.transforms import v2
from torchvision.models.detection.anchor_utils import AnchorGenerator
from torchvision.models.detection.image_list import ImageList
from torchvision.models.detection._utils import Matcher, BoxCoder
from torch.utils.data import Dataset
from PIL import Image

In [2]:
def transform_img(image: Image.Image) -> torch.Tensor:
  tr = v2.Compose([
    v2.PILToTensor(),
    v2.Resize((384, 384)),
    v2.ToDtype(torch.float, True),
  ])
  return tr(image)

class CarPlateTrainDataset(Dataset):

  def __init__(self, path: str, compact: bool =False, images=None, labels=None) -> None:
    super().__init__()
    self.compact = compact
    if compact:
      self.images = images
      self.labels = labels
    else:
      self.path = path + 'train/'
      self.train = []
      with open(path + 'train.txt', 'r') as f:
        self.train = [x.rstrip('\n') for x in f]

  def __len__(self) -> int:
    if self.compact:
      return self.images.size(0)
    else:
      return len(self.train)

  def __getitem__(self, item: int) -> tuple[torch.Tensor, torch.Tensor]:
    if self.compact:
      return self.images[item], self.labels[item]
    else:
      image_path = self.path + self.train[item] + '.jpg'
      label_path = self.path + self.train[item] + '.json'
      image = Image.open(image_path)
      w, h = image.size
      scale_x = 384 / w
      scale_y = 384 / h
      image = transform_img(image)
      with open(label_path) as f:
        full_label = json.load(f)
      labels = []
      for lbl in full_label['lps']:
        lp = torch.Tensor(lbl['poly_coord'])
        x_min = lp[:, 0].min() * scale_x
        y_min = lp[:, 1].min() * scale_y
        x_max = lp[:, 0].max() * scale_x
        y_max = lp[:, 1].max() * scale_y
        labels.append(torch.tensor([x_min, y_min, x_max, y_max], dtype=torch.float32, device=device))
      labels = torch.stack(labels)
      return image.to(device), labels

class CarPlateTestDataset(Dataset):

  def __init__(self, path: str) -> None:
    super().__init__()
    self.path = path + 'test/'
    self.test = []
    with open(path+'test.txt', 'r') as f:
      self.test = [x.rstrip('\n') for x in f]

  def __len__(self) -> int:
    return len(self.test)

  def __getitem__(self, item: int) -> tuple[torch.Tensor, torch.Tensor]:
    image_path = self.path + self.test[item] + '.jpg'
    label_path = self.path + self.test[item] + '.json'
    image = Image.open(image_path)
    w, h = image.size
    scale_x = 384 / w
    scale_y = 384 / h
    image = transform_img(image)
    with open(label_path) as f:
      full_label = json.load(f)
    labels = []
    for lbl in full_label['lps']:
      lp = torch.Tensor(lbl['poly_coord'])
      x_min = lp[:, 0].min() * scale_x
      y_min = lp[:, 1].min() * scale_y
      x_max = lp[:, 0].max() * scale_x
      y_max = lp[:, 1].max() * scale_y
      labels.append(torch.tensor([x_min, y_min, x_max, y_max], dtype=torch.float32, device=device))
    labels = torch.stack(labels)
    return image.to(device), labels

In [51]:
def detection_loss_3v1(cls_preds: torch.Tensor, reg_preds: torch.Tensor, anchors: list[torch.Tensor], gt_boxes: list[torch.Tensor], matcher: Matcher, box_coder: BoxCoder, neg_pos_ratio: int = 3):
  batch_size = cls_preds.size(0)
  cls_loss = 0.0
  reg_loss = 0.0

  for i in range(batch_size):
    # Match anchors with GT
    iou_matrix = box_iou(gt_boxes[i], anchors[i])
    matched_idxs = matcher(iou_matrix)
    matched_mask = matched_idxs >= 0

    # Classification targets: 1 for positive anchors, 0 for negatives
    labels = torch.zeros(cls_preds[i].size(0), dtype=torch.long, device=cls_preds.device)
    labels[matched_mask] = 1

    """max_iou_per_anchor, _ = iou_matrix.max(dim=0)
    print("Max IoU anchors:", (max_iou_per_anchor > 0.5).sum().item())
    print("Max IoU > 0.3:", (max_iou_per_anchor > 0.3).sum().item())
    print("Max IoU (max):", max_iou_per_anchor.max().item())"""

    """aw = anchors[i][:, 2] - anchors[i][:, 0]
    ah = anchors[i][:, 3] - anchors[i][:, 1]
    print(anchors[i][20:40])
    print("Anchor width mean:", aw.mean().item())
    print("Anchor height mean:", ah.mean().item())
    aw = gt_boxes[i][:, 2] - gt_boxes[i][:, 0]
    ah = gt_boxes[i][:, 3] - gt_boxes[i][:, 1]
    print("GT box:", gt_boxes[i])
    print("GT width mean:", aw.mean().item())
    print("GT height mean:", ah.mean().item())
    print("Cls logits mean:", cls_preds.mean().item())
    print("Cls logits std:", cls_preds.std().item())"""

    # Hard negative mining
    num_pos = matched_mask.sum().item()
    if num_pos > 0:
      # Probabilities of being positive (before softmax)
      probs = F.softmax(cls_preds[i], dim=1)[:, 1]  # probability of class 1 (license plate)
      # Only consider negatives
      neg_probs = probs[labels == 0]
      # Take top-k hardest negatives
      k = min(neg_pos_ratio * num_pos, neg_probs.numel())
      if k > 0:
        topk_vals, topk_idx = torch.topk(neg_probs, k)
        hard_neg_mask = torch.zeros_like(labels, dtype=torch.bool)
        neg_idx_in_all = (labels == 0).nonzero(as_tuple=True)[0]
        hard_neg_mask[neg_idx_in_all[topk_idx]] = True
      else:
        hard_neg_mask = torch.zeros_like(labels, dtype=torch.bool)


    else:
      hard_neg_mask = torch.zeros_like(labels, dtype=torch.bool)    # Combine positives and hard negatives

    cls_mask = matched_mask | hard_neg_mask
    cls_loss += F.cross_entropy(cls_preds[i][cls_mask], labels[cls_mask])

    # Regression loss only for positives
    if matched_mask.sum() > 0:
      matched_gt_boxes = gt_boxes[i][matched_idxs[matched_mask]]
      matched_anchors = anchors[i][matched_mask]
      encoded_targets = box_coder.encode([matched_anchors], [matched_gt_boxes])[0]
      reg_loss += F.smooth_l1_loss(reg_preds[i][matched_mask], encoded_targets)

    return cls_loss + reg_loss



def detection_loss(cls_preds: torch.Tensor, reg_preds: torch.Tensor, anchors: list[torch.Tensor], gt_boxes: list[torch.Tensor], matcher: Matcher, box_coder: BoxCoder):
  batch_size = cls_preds.size(0)
  cls_loss = 0.0
  reg_loss = 0.0

  for i in range(batch_size):
    # Match anchors with GT
    iou_matrix = box_iou(gt_boxes[i], anchors[i])
    matched_idxs = matcher(iou_matrix)
    matched_mask = matched_idxs >= 0

    """max_iou_per_anchor, _ = iou_matrix.max(dim=0)
    print("Max IoU anchors:", (max_iou_per_anchor > 0.5).sum().item())
    print("Max IoU > 0.3:", (max_iou_per_anchor > 0.3).sum().item())
    print("Max IoU (max):", max_iou_per_anchor.max().item())
    """
    """aw = anchors[i][:, 2] - anchors[i][:, 0]
    ah = anchors[i][:, 3] - anchors[i][:, 1]
    print(anchors[i][10:20])
    print("Anchor width mean:", aw.mean().item())
    print("Anchor height mean:", ah.mean().item())
    aw = gt_boxes[i][:, 2] - gt_boxes[i][:, 0]
    ah = gt_boxes[i][:, 3] - gt_boxes[i][:, 1]
    print(gt_boxes[i])
    print("GT width mean:", aw.mean().item())
    print("GT height mean:", ah.mean().item())
    print("Cls logits mean:", cls_preds.mean().item())
    print("Cls logits std:", cls_preds.std().item())"""


    # Classification loss
    labels = torch.zeros_like(cls_preds[i][:, 0], dtype=torch.long)
    labels[matched_mask] = 1
    cls_loss += F.cross_entropy(cls_preds[i], labels)

    # Regression loss for matched anchors
    if matched_mask.sum() > 0:
      matched_gt_boxes = gt_boxes[i][matched_idxs[matched_mask]]
      matched_anchors = anchors[i][matched_mask]
      encoded_targets = box_coder.encode([matched_anchors], [matched_gt_boxes])[0]
      reg_loss += F.smooth_l1_loss(reg_preds[i][matched_mask], encoded_targets)

  return cls_loss + reg_loss

In [48]:
class InResBlock(nn.Module):
  def __init__(self, inc, outc):
    super().__init__()

    self.branch1 = nn.Sequential(
      nn.Conv2d(inc, outc, kernel_size=1, stride=1, padding=0),
      nn.ReLU()
    )
    self.branch2 = nn.Sequential(
      nn.Conv2d(inc, outc, kernel_size=3, stride=1, padding=1),
      nn.ReLU(),
      nn.Conv2d(outc, outc, kernel_size=1, stride=1, padding=0),
      nn.ReLU()
    )
    self.branch3 = nn.Sequential(
      nn.Conv2d(inc, outc, kernel_size=5, stride=1, padding=2),
      nn.ReLU(),
      nn.Conv2d(outc, outc, kernel_size=3, stride=1, padding=1),
      nn.ReLU(),
      nn.Conv2d(outc, outc, kernel_size=1, stride=1, padding=0),
      nn.ReLU()
    )

    self.merger = nn.Conv2d(3 * outc, inc, kernel_size=1)

    self.relu = nn.ReLU()

  def forward(self, x):
    b1 = self.branch1(x)
    b2 = self.branch2(x)
    b3 = self.branch3(x)

    merged = torch.cat([b1, b2, b3], dim=1)

    out = self.merger(merged) + x
    out = self.relu(out)
    return out

class Prediction(nn.Module):

  def __init__(self, inc: int, num_anchors: int) -> None:
    super().__init__()
    self.cls = nn.Conv2d(inc, 2 * num_anchors, kernel_size=3, padding=1) # (32, 16, 16) -> (2*num_anchors, 16, 16) or (48, 8, 8) -> (2*num_anchors, 8, 8)
    self.reg = nn.Conv2d(inc, 4 * num_anchors, kernel_size=3, padding=1) # (32, 16, 16) -> (4*num_anchors, 16, 16) or (48, 8, 8) -> (4*num_anchors, 8, 8)

  def forward(self, x:torch.Tensor) -> tuple[torch.Tensor, torch.Tensor]:
    cls = self.cls(x)
    reg = self.reg(x)

    cls = cls.permute(0, 2, 3, 1).reshape(cls.size(0), -1, 2) # (2*num_anchors, 16, 16) -> (256*num_anchors, 2) or (2*num_anchors, 8, 8) -> (64*num_anchors, 2)
    reg = reg.permute(0, 2, 3, 1).reshape(reg.size(0), -1, 4) # (4*num_anchors, 16, 16) -> (256*num_anchors, 4) or (4*num_anchors, 8, 8) -> (64*num_anchors, 4)

    return cls, reg

class LPD(nn.Module):

  def __init__(self) -> None:
    super().__init__()

    self.block1 = nn.Sequential(
      nn.Conv2d(3, 24, 5, padding=2), # (3, 384, 384) -> (16, 128, 128)
      nn.ReLU(),
      InResBlock(24, 24) #(16, 128, 128) -> (16, 128, 128)
    )
    self.block2 = nn.Sequential(
      nn.Conv2d(24, 32, 3, stride=2, padding=1),  # (16, 128, 128) -> (24, 64, 64)
      nn.ReLU(),
      InResBlock(32, 32)  # (24, 64, 64) -> (24, 64, 64)
    )
    self.block3 = nn.Sequential(
      nn.Conv2d(32, 48, 3, stride=2, padding=1),  # (24, 128, 128) -> (32, 32, 32)
      nn.ReLU(),
      InResBlock(48, 48) # (32, 32, 32) -> (32, 32, 32)
    )
    self.block4 = nn.Sequential(
      nn.Conv2d(48, 48, 3, stride=2, padding=1),  # (32, 32, 32) -> (32, 16, 16)
      nn.ReLU(),
      InResBlock(48, 48) # (32, 16, 16) -> (32, 16, 16)
    )
    self.block5 = nn.Sequential(
      nn.Conv2d(48, 48, 3, stride=2, padding=1),  # (32, 16, 16) -> (48, 8, 8)
      nn.ReLU(),
      InResBlock(48, 48)  # (48, 8, 8) -> (48, 8, 8)
    )

    self.FM0 = Prediction(48, num_anchors=10) #(32, 16, 16) -> ((2*num_anchors, 16, 16) # LP/no LP, or (4*num_anchors, 16, 16) # (minx, miny, maxx, maxy))
    self.FM1 = Prediction(48, num_anchors=10) #(48, 8, 8) -> ((2*num_anchors, 8, 8)# LP/no LP, or (4*num_anchors, 8, 8) # (minx, miny, maxx, maxy))
    self.FM2 = Prediction(48, num_anchors=10)  # (48, 8, 8) -> ((2*num_anchors, 8, 8)# LP/no LP, or (4*num_anchors, 8, 8) # (minx, miny, maxx, maxy))

    self.anchors = AnchorGenerator(sizes=((40, 60), (80, 100), (120, 140)), aspect_ratios=((0.25, 0.33, 0.66, 1.0, 2.0),) * 3)

  def forward(self, x: torch.Tensor) -> tuple[torch.Tensor, torch.Tensor, list[torch.Tensor]]:
    images = x
    batch, channels, height, width = images.size()
    x = self.block1(x)
    x = self.block2(x)
    fm0 = self.block3(x)
    fm1 = self.block4(fm0)
    fm2 = self.block5(fm1)

    image_list = ImageList(images, [(height, width)] * batch)
    anchors = self.anchors(image_list, [fm0, fm1, fm2])

    cls0, reg0 = self.FM0(fm0)
    cls1, reg1 = self.FM1(fm1)
    cls2, reg2 = self.FM2(fm2)

    cls = torch.cat([cls0, cls1, cls2], dim=1)
    reg = torch.cat([reg0, reg1, reg2], dim=1)

    return cls, reg, anchors

In [49]:
model = LPD().to(device)

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
#Model loading

Using device: cuda


In [23]:
import gc
torch.cuda.empty_cache()
gc.collect()

682

In [6]:
def collate_fn(batch: list) -> tuple[torch.Tensor, list[torch.Tensor]]:
  imgs, labels = zip(*batch)
  imgs = torch.stack(imgs)
  labels = list(labels)
  return imgs, labels

dataset = CarPlateTrainDataset('drive/MyDrive/dataset/')
images_dataset = []
lbls_dataset = []
gt_widths = []
gt_heights = []

for img, label in dataset:
  images_dataset.append(img)
  lbls_dataset.append(label)

images_dataset = torch.stack(images_dataset)

print(images_dataset.device)
print(lbls_dataset[50].device)

train_dataset = CarPlateTrainDataset('drive/MyDrive/dataset/', compact=True, images=images_dataset, labels=lbls_dataset)
train_loader = DataLoader(train_dataset, batch_size=32, collate_fn=collate_fn)

cuda:0
cuda:0


In [7]:
def collate_fn(batch: list) -> tuple[torch.Tensor, list[torch.Tensor]]:
  imgs, labels = zip(*batch)
  imgs = torch.stack(imgs)
  labels = list(labels)
  return imgs, labels

model = LPD().to(device)
matcher = Matcher(0.5, 0.3, allow_low_quality_matches=True)
box_coder = BoxCoder((1., 1., 1., 1.))
optimizer = AdamW(model.parameters(), lr=0.0002, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

train_dataset = CarPlateTrainDataset('drive/MyDrive/dataset/', compact=True, images=images_dataset, labels=lbls_dataset)
train_loader = DataLoader(train_dataset, batch_size=16, collate_fn=collate_fn)
#test_dataset = CarPlateTestDataset('drive/MyDrive/dataset/')
#test_loader = DataLoader(test_dataset, batch_size=64, collate_fn=collate_fn)

In [ ]:
checkpoint = torch.load('model.pth', weights_only=True, map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
last_epoch = checkpoint['epoch']
loss = checkpoint['loss']
print(f'Checkpoint loaded. Last epoch: {last_epoch}, with loss: {loss}')

FileNotFoundError: [Errno 2] No such file or directory: 'model.pth'

In [53]:
last_epoch = 0
model.train()
num_epochs = 5 + last_epoch
for epoch in range(last_epoch+1, num_epochs+1):
  loop = tqdm(train_loader, desc=f"Epoch {epoch}/{num_epochs}")
  for images, targets in loop:
    cls_preds, reg_preds, anchors = model(images)
    loss = detection_loss(cls_preds, reg_preds, anchors, targets, matcher, box_coder)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    scheduler.step()
    loop.set_postfix(loss=loss.item())


Epoch 1/5:   0%|          | 0/99 [00:00<?, ?it/s]

Epoch 2/5:   0%|          | 0/99 [00:00<?, ?it/s]

Epoch 3/5:   0%|          | 0/99 [00:00<?, ?it/s]

Epoch 4/5:   0%|          | 0/99 [00:00<?, ?it/s]

Epoch 5/5:   0%|          | 0/99 [00:00<?, ?it/s]

In [18]:
fallo = 0
for img, label in train_loader:
  cls_preds, reg_preds, anchors = model(img)
  for i in range(cls_preds.size(0)):
    """ws = anchors[i][:, 2] - anchors[i][:, 0]
    hs = anchors[i][:, 3] - anchors[i][:, 1]
    l = torch.stack([ws,hs])
    l = torch.unique(l, dim=1)
    print(l)
    ws = label[i][:, 2] - label[i][:, 0]
    hs = label[i][:, 3] - label[i][:, 1]
    l = torch.stack([ws,hs])
    l = torch.unique(l, dim=1)
    print(l)"""

    iou_matrix = box_iou(anchors[i], label[i])
    print(iou_matrix.max().item())
    matched_idxs = matcher(iou_matrix)
    print(matched_idxs)
    print('anchor ajustado: ', anchors[i][matched_idxs])
    print('GT: ', label[i])
    if iou_matrix.max().item() == 0: fallo += 1
    print()

  print(f'fallo: {fallo/64}')
  break

0.7640432119369507
tensor([58290], device='cuda:0')
anchor ajustado:  tensor([[139., 137., 229., 183.]], device='cuda:0')
GT:  tensor([[154.6667, 138.8642, 228.8889, 184.9877]], device='cuda:0')

0.4338615834712982
tensor([48213, 40227], device='cuda:0')
anchor ajustado:  tensor([[152., 100., 216., 164.],
        [338.,  53., 374., 163.]], device='cuda:0')
GT:  tensor([[152.9630, 130.3704, 212.7407, 160.0988],
        [344.8889, 142.1235, 370.3704, 161.1852]], device='cuda:0')

0.44453009963035583
tensor([65553, 19932], device='cuda:0')
anchor ajustado:  tensor([[168., 148., 232., 212.],
        [302.,  -3., 338., 107.]], device='cuda:0')
GT:  tensor([[176.0741, 175.2099, 230.2963, 208.7901],
        [309.1111,  89.6790, 337.0370, 106.4691]], device='cuda:0')

0.5816010236740112
tensor([74055], device='cuda:0')
anchor ajustado:  tensor([[119., 181., 209., 227.]], device='cuda:0')
GT:  tensor([[135.4074, 191.0123, 207.1111, 224.5926]], device='cuda:0')

0.14860709011554718
tensor([45318

In [54]:
epoch = 1
num_epochs = 1
iter = 0
with torch.no_grad():
  loop = tqdm(train_loader, desc=f"Epoch {epoch}/{num_epochs}")
  for images, targets in loop:
    iter += 1
    if iter == 20:
      break
    cls_preds, reg_preds, anchors = model(images)
    batch_size = cls_preds.size(0)

    for i in range(batch_size):
      # Match anchors with GT

      iou_matrix = box_iou(targets[i], anchors[i])
      matched_idxs = matcher(iou_matrix)
      matched_mask = matched_idxs >= 0

      max_iou_per_anchor, _ = iou_matrix.max(dim=0)
      print("Max IoU anchors:", (max_iou_per_anchor > 0.5).sum().item())
      print("Max IoU > 0.3:", (max_iou_per_anchor > 0.3).sum().item())
      print("Max IoU (max):", max_iou_per_anchor.max().item())

      """aw = anchors[i][:, 2] - anchors[i][:, 0]
      ah = anchors[i][:, 3] - anchors[i][:, 1]
      print(anchors[i][20:40])
      print("Anchor width mean:", aw.mean().item())
      print("Anchor height mean:", ah.mean().item())
      aw = gt_boxes[i][:, 2] - gt_boxes[i][:, 0]
      ah = gt_boxes[i][:, 3] - gt_boxes[i][:, 1]
      print("GT box:", gt_boxes[i])
      print("GT width mean:", aw.mean().item())
      print("GT height mean:", ah.mean().item())
      print("Cls logits mean:", cls_preds.mean().item())
      print("Cls logits std:", cls_preds.std().item())"""

Epoch 1/1:   0%|          | 0/99 [00:00<?, ?it/s]

Max IoU anchors: 166
Max IoU > 0.3: 1540
Max IoU (max): 0.9161298274993896
Max IoU anchors: 114
Max IoU > 0.3: 894
Max IoU (max): 0.7815302610397339
Max IoU anchors: 113
Max IoU > 0.3: 888
Max IoU (max): 0.8787368535995483
Max IoU anchors: 191
Max IoU > 0.3: 1141
Max IoU (max): 0.6809468865394592
Max IoU anchors: 0
Max IoU > 0.3: 119
Max IoU (max): 0.3952562212944031
Max IoU anchors: 0
Max IoU > 0.3: 0
Max IoU (max): 0.16570641100406647
Max IoU anchors: 106
Max IoU > 0.3: 672
Max IoU (max): 0.844704806804657
Max IoU anchors: 202
Max IoU > 0.3: 1147
Max IoU (max): 0.6977347731590271
Max IoU anchors: 84
Max IoU > 0.3: 638
Max IoU (max): 0.8086827993392944
Max IoU anchors: 161
Max IoU > 0.3: 982
Max IoU (max): 0.6684027910232544
Max IoU anchors: 63
Max IoU > 0.3: 601
Max IoU (max): 0.8959442377090454
Max IoU anchors: 0
Max IoU > 0.3: 0
Max IoU (max): 0.28515610098838806
Max IoU anchors: 32
Max IoU > 0.3: 206
Max IoU (max): 0.6345130801200867
Max IoU anchors: 72
Max IoU > 0.3: 682
Max IoU 

In [60]:
model.eval()
total_gt = 0
correct_detections = 0
iter = 0
with torch.no_grad():
  for images, targets in train_loader:
    iter += 1
    if iter == 10:
      break
    cls_preds, reg_preds, anchors = model(images)
    batch_size = images.size(0)

    for i in range(batch_size):

      gt_boxes = targets[i]
      total_gt += len(gt_boxes)
      scores = cls_preds[i].softmax(dim=1)[:, 1]

      mask = scores > 0.54

      if mask.sum() == 0:
        continue

      # Decodificar cajas
      print('anchors y cajas predichas antes de codificar')
      print('gt', targets[i])
      print('anchors', anchors[i][mask])
      print('pred', reg_preds[i][mask])
      pred_boxes = box_coder.decode(anchors[i][mask], [reg_preds[i][mask]]).reshape(1, -1, 4).squeeze(0)
      pred_scores = scores[mask]
      print('cajas predichas y probabilidades despues de codificar')
      print('pred', pred_boxes)
      print(pred_scores)

      # NMS
      keep = nms(pred_boxes, pred_scores, iou_threshold=0.5)
      pred_boxes = pred_boxes[keep]
      print('cajas predichas tras nms')
      print(pred_boxes)
      print()

      # Comparar cada GT con las predicciones
      if len(pred_boxes) > 0:
        iou = box_iou(gt_boxes, pred_boxes)
        max_iou_per_gt, _ = iou.max(dim=1)

        # Contar GT detectadas con IoU suficiente
        correct_detections += (max_iou_per_gt >= 0.5).sum().item()
recall = correct_detections / total_gt if total_gt > 0 else 0
print("Recall:", recall)
print(f"Accuracy of detection = {recall * 100:.2f}%")

anchors y cajas predichas antes de codificar
gt tensor([[132.7407, 212.3457, 194.1481, 233.8765],
        [279.6296, 107.7531, 302.2222, 117.1358]], device='cuda:0')
anchors tensor([[130., 182., 190., 242.]], device='cuda:0')
pred tensor([[ 0.0341,  0.0283, -0.0112,  0.1085]], device='cuda:0')
cajas predichas y probabilidades despues de codificar
pred tensor([[-4.4591, 12.1685, -7.2886, 17.1851]], device='cuda:0')
tensor([0.5404], device='cuda:0')
cajas predichas tras nms
tensor([[-4.4591, 12.1685, -7.2886, 17.1851]], device='cuda:0')

anchors y cajas predichas antes de codificar
gt tensor([[200.8889, 274.9630, 218.3704, 282.3704]], device='cuda:0')
anchors tensor([[302.,  42., 362., 102.],
        [306.,  62., 366., 122.]], device='cuda:0')
pred tensor([[0.0369, 0.0321, 0.0112, 0.0953],
        [0.0373, 0.0340, 0.0123, 0.0951]], device='cuda:0')
cajas predichas y probabilidades despues de codificar
pred tensor([[-6.9400,  0.7428, -8.5476,  4.6909],
        [-6.8466,  1.9461, -8.4096, 